In [28]:
import torch
torch.set_printoptions(profile='full')


In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# ============================================================================
# 1) OPTConfig: 모델 하이퍼파라미터 저장
# ============================================================================
class OPTConfig:
    def __init__(
        self,
        vocab_size=50272,
        hidden_size=768,
        num_hidden_layers=12,
        num_attention_heads=12,
        max_position_embeddings=2048,
        ffn_dim=None,
        dropout=0.1,
        layer_norm_epsilon=1e-5,
    ):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads
        self.max_position_embeddings = max_position_embeddings
        self.ffn_dim = ffn_dim or 4 * hidden_size
        self.dropout = dropout
        self.layer_norm_epsilon = layer_norm_epsilon


# ============================================================================
# 2) OPTAttention: 분리형 Q/K/V 기반의 causal self-attention
# ============================================================================
class OPTAttention(nn.Module):
    def __init__(self, config: OPTConfig):
        super().__init__()
        self.num_heads = config.num_attention_heads
        self.head_dim = config.head_dim
        self.scale = 1.0 / math.sqrt(self.head_dim)

        # Q, K, V projections (bias=True로 변경)
        self.q_proj = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.k_proj = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        self.v_proj = nn.Linear(config.hidden_size, config.hidden_size, bias=True)
        # output projection
        self.out_proj = nn.Linear(config.hidden_size, config.hidden_size, bias=True)

        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)

        # causal mask buffer
        mask = torch.tril(torch.ones(config.max_position_embeddings, config.max_position_embeddings))
        self.register_buffer('mask', mask)

    def forward(self, x, attention_mask=None):
        B, T, C = x.size()
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        # reshape to heads
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2) # B, num_heads, T, head_dim
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        # attention scores
        attn_scores = (q @ k.transpose(-2, -1)) * self.scale # B, num_heads, T, head_dim @ B, num_heads, head_dim, T
        causal = self.mask[:T, :T].unsqueeze(0).unsqueeze(0)
        attn_scores = attn_scores.masked_fill(causal == 0, float('-inf'))
        if attention_mask is not None:
            attn_scores = attn_scores + attention_mask
        attn_probs = F.softmax(attn_scores, dim=-1)
        attn_probs = self.attn_dropout(attn_probs)
        out = attn_probs @ v
        out = out.transpose(1, 2).reshape(B, T, C)
        out = self.out_proj(out)
        out = self.resid_dropout(out)
        return out


# ============================================================================
# 3) FeedForward (MLP)
# ============================================================================
class OPTFeedForward(nn.Module):
    def __init__(self, config: OPTConfig):
        super().__init__()
        self.fc1 = nn.Linear(config.hidden_size, config.ffn_dim, bias=True)
        self.activation = F.silu
        self.fc2 = nn.Linear(config.ffn_dim, config.hidden_size, bias=True)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return self.dropout(x)


# ============================================================================
# 4) OPTDecoderLayer: 한 레이어
# ============================================================================
class OPTDecoderLayer(nn.Module):
    def __init__(self, config: OPTConfig):
        super().__init__()
        self.self_attn = OPTAttention(config)
        self.layernorm1 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.feed_forward = OPTFeedForward(config)
        self.layernorm2 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, x, attention_mask=None):
        r = x
        x = self.layernorm1(x)
        x = self.self_attn(x, attention_mask)
        x = x + r
        r = x
        x = self.layernorm2(x)
        x = self.feed_forward(x)
        return x + r


# ============================================================================
# 5) OPTModel: 임베딩 + 디코더 스택
# ============================================================================
class OPTModel(nn.Module):
    def __init__(self, config: OPTConfig):
        super().__init__()
        self.token_embedding = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.dropout = nn.Dropout(config.dropout)
        self.layers = nn.ModuleList([OPTDecoderLayer(config) for _ in range(config.num_hidden_layers)])
        self.final_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, input_ids, attention_mask=None):
        B, T = input_ids.size()
        device = input_ids.device
        positions = torch.arange(T, device=device).unsqueeze(0).expand(B, T)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x, attention_mask)
        return self.final_layernorm(x)


# ============================================================================
# 6) OPTForCausalLM: LM 헤드 (bias=True로 변경)
# ============================================================================
class OPTForCausalLM(nn.Module):
    def __init__(self, config: OPTConfig):
        super().__init__()
        self.model = OPTModel(config)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=True)

    def forward(self, input_ids, attention_mask=None):
        hidden = self.model(input_ids, attention_mask)
        return self.lm_head(hidden)


# ============================================================================
# 7) 가중치 로드 및 테스트
# ============================================================================
if __name__ == '__main__':
    # config 생성
    config = OPTConfig()
    from transformers import OPTForCausalLM as HF_OPT, OPTConfig as HFConfig

    # Hugging Face 모델과 체크포인트
    hf_cfg = HFConfig.from_pretrained('facebook/opt-125m')
    hf_model = HF_OPT.from_pretrained('facebook/opt-125m', config=hf_cfg)
    hf_state = hf_model.state_dict()

    # 커스텀 모델 생성
    model = OPTForCausalLM(config)

    # 새로운 state_dict 매핑 및 버퍼 추가
    new_state = {}
    for k, v in hf_state.items():
        if k == 'model.decoder.embed_tokens.weight':
            new_state['model.token_embedding.weight'] = v.clone()
        elif k == 'model.decoder.embed_positions.weight':
            new_state['model.position_embedding.weight'] = v[:config.max_position_embeddings].clone()
        elif k.startswith('model.decoder.layers.'):
            parts = k.split('.')
            idx = parts[3]
            sub = parts[4:]
            base = f'model.layers.{idx}'
            if sub[0] == 'self_attn':
                name = sub[1] + '.' + sub[2]
                new_state[f'{base}.self_attn.{name}'] = v.clone()
            elif sub[0] == 'self_attn_layer_norm':
                new_state[f'{base}.layernorm1.{sub[1]}'] = v.clone()
            elif sub[0] == 'fc1':
                new_state[f'{base}.feed_forward.fc1.{sub[1]}'] = v.clone()
            elif sub[0] == 'fc2':
                new_state[f'{base}.feed_forward.fc2.{sub[1]}'] = v.clone()
            elif sub[0] == 'final_layer_norm':
                new_state[f'{base}.layernorm2.{sub[1]}'] = v.clone()
        elif k == 'model.decoder.final_layer_norm.weight':
            new_state['model.final_layernorm.weight'] = v.clone()
        elif k == 'model.decoder.final_layer_norm.bias':
            new_state['model.final_layernorm.bias'] = v.clone()
        elif k == 'lm_head.weight':
            new_state['lm_head.weight'] = v.clone()
        elif k == 'lm_head.bias':
            new_state['lm_head.bias'] = v.clone()
    # buffer mask
    for i in range(config.num_hidden_layers):
        mask_buf = model.model.layers[i].self_attn.mask
        new_state[f'model.layers.{i}.self_attn.mask'] = mask_buf.clone()

    # lm_head.bias가 체크포인트에 없으면 0으로 초기화
    if 'lm_head.bias' not in new_state:
        new_state['lm_head.bias'] = torch.zeros_like(model.lm_head.bias)

    missing, unexpected = model.load_state_dict(new_state, strict=False)
    print('Missing keys:', missing)
    print('Unexpected keys:', unexpected)

    # 테스트
    dummy = torch.randint(0, config.vocab_size, (1, 5))
    out = model(dummy)
    print('out shape:', out.shape)  # (1,5,50272)


/root/miniconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/myenv/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/envs/myenv/lib/python3.8/site-packages/torchvisio

Missing keys: []
Unexpected keys: []
out shape: torch.Size([1, 5, 50272])


Error during conversion: ChunkedEncodingError(ProtocolError("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)))


In [4]:
model

OPTForCausalLM(
  (model): OPTModel(
    (token_embedding): Embedding(50272, 768)
    (position_embedding): Embedding(2048, 768)
    (dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (layernorm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (feed_forward): OPTFeedForward(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
   

In [9]:
model.model.layers[0].self_attn.q_proj

Linear(in_features=768, out_features=768, bias=True)

In [17]:
y=torch.tensor([[1,2,3,4],[5,6,7,8]]).reshape(4,2)
print(y)

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])


In [21]:
y=torch.randn(1, 768, 1)
print(y)
y= y.reshape(64, 12).reshape(-1, 768)
print(y)

tensor([[[ 5.1766e-01],
         [ 2.3710e-01],
         [-3.1125e-01],
         [-9.7572e-02],
         [-1.2121e+00],
         [ 6.6000e-01],
         [ 5.5576e-01],
         [-7.6201e-01],
         [-1.7650e+00],
         [-3.1692e-01],
         [-1.0299e-01],
         [-1.0987e+00],
         [ 1.8297e-01],
         [ 4.2828e-01],
         [ 1.5890e+00],
         [-1.2110e+00],
         [ 6.7471e-01],
         [ 2.3961e-01],
         [-2.0213e+00],
         [ 7.1468e-01],
         [ 2.2473e+00],
         [-1.2804e-01],
         [-3.4481e-01],
         [ 1.1961e+00],
         [-5.5442e-01],
         [ 1.0019e+00],
         [-1.5590e+00],
         [-6.3782e-01],
         [ 1.2268e+00],
         [ 1.1523e+00],
         [ 2.0763e-01],
         [-2.5591e-01],
         [-7.3185e-03],
         [ 1.2510e+00],
         [ 1.3957e+00],
         [-9.8795e-01],
         [ 6.8271e-01],
         [-3.5732e-01],
         [ 2.6001e-02],
         [ 5.1747e-01],
         [ 1.4850e+00],
         [ 1.469

In [35]:
model.model.layers[0].self_attn.q_proj.weight.shape

torch.Size([768, 768])

In [ ]:
import torch
batch=1
hidden=768
sequence_length=128
new_token_length=1
num_head=12
# init memory
act_buffer = torch.zeros(256, 12)    # (your real buffer)
k_cache = torch.zeros(64*128, 12)
v_cache = torch.zeros(64*128, 12)

# init new token
new_token = torch.randn(batch, hidden, new_token_length)
act_buffer[0:64, :]=new_token.reshape(64, 12)


## ===========
## Q projection
## ============

W = model.model.layers[0].self_attn.q_proj.weight    
b = model.model.layers[0].self_attn.q_proj.bias      

W = W.T   

# 3. Define your split parameters
num_pe = 96
num_acc = int(hidden/num_pe) # 32

# 4. Allocate y with the correct output size
x = act_buffer[0:64, :].reshape(-1, 768)
y = torch.zeros(1, hidden)

# 5. Loop over each of the 2304 output rows, dot‐product in 32 pieces
for j in range(hidden):
    acc = 0.0
    for i in range(num_acc):
        start = i * num_pe
        end   = start + num_pe
        # x[:, start:end] is [1×24], W[j, start:end] is [24]
        acc += (x[:, start:end] * W[j, start:end]).sum()
    y[0, j] = acc + b[j]
act_buffer[0:64, :]=y.reshape(64, 12)


## =====
## K projection 
## =====

W = model.model.layers[0].self_attn.k_proj.weight    
b = model.model.layers[0].self_attn.k_proj.bias      

W = W.T 
# 4. Allocate y with the correct output size
x = act_buffer[0:64, :].reshape(-1, 768)
y = torch.zeros(1, hidden)

# 5. Loop over each of the 2304 output rows, dot‐product in 32 pieces
for j in range(hidden):
    acc = 0.0
    for i in range(num_acc):
        start = i * num_pe
        end   = start + num_pe
        # x[:, start:end] is [1×24], W[j, start:end] is [24]
        acc += (x[:, start:end] * W[j, start:end]).sum()
    y[0, j] = acc + b[j]
k_cache[0:64, :]=y.reshape(64, 12)



## =====
## V projection 
## =====

W = model.model.layers[0].self_attn.v_proj.weight    
b = model.model.layers[0].self_attn.v_proj.bias      

W = W.T 
# 4. Allocate y with the correct output size
x = act_buffer[0:64, :].reshape(-1, 768)
y = torch.zeros(1, hidden)

# 5. Loop over each of the 2304 output rows, dot‐product in 32 pieces
for j in range(hidden):
    acc = 0.0
    for i in range(num_acc):
        start = i * num_pe
        end   = start + num_pe
        # x[:, start:end] is [1×24], W[j, start:end] is [24]
        acc += (x[:, start:end] * W[j, start:end]).sum()
    y[0, j] = acc + b[j]
v_cache[0:64, :]=y.reshape(64, 12)

## ====
## attention score
## ====
num_token=3
y=torch.zeros(num_token, num_head)
for j in range(num_token):
    for i in range(64):
        for k in range(num_head):
            y[j:j+1, k:k+1]+=act_buffer[i,k:k+1 ] * k_cache[i+num_token*64, k:k+1]
y=F.softmax(y, dim=0)
act_buffer[0:num_token, 0:num_head] = y #(num_token, num_head)

## === 
## attention score * V
## ===
for i in range(num_token):
     

In [73]:
k_cache[64]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<SelectBackward0>)